import modules

In [137]:
import pandas as pd
import glob
import os
current_directory = os.getcwd()
print("현재 디렉토리:", current_directory)

현재 디렉토리: c:\apps\Project1


목표 : 각 옵션종목을 각 dataframe으로 저장
dataframe이름 예로 201TB305는 콜옵션, 23년, 11월, 행사가 305이므로 -> TB305C
201TC2337는 콜옵션, 23년, 12월, 행사가 337.5이므로 -> TC337C
301TB312는 풋옵션, 23년, 11월, 행사가 312.5이므로 -> TB312P
301TC335는 풋옵션, 23년, 12월, 행사가 335이므로 -> TC335P

load files

In [128]:
strike_price = pd.read_csv('./dataset/strike_price.csv')

In [ ]:
strike_price